**IMPORT NECESSARY PACKAGES**

In [1]:
## Importing libraries
import pandas as pd
import numpy as np
import cv2
import os
from tqdm import tqdm
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow as tf
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Flatten, Dense, Conv2D, Dropout, Input, AveragePooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from keras.utils import to_categorical

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

**BUILDING MODEL**

In [2]:

#Using pre-trained model
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

9406464/9406464 [==============================] - 1s 0us/step


In [3]:
#Define class Model base on architecture of VGG16
class Model(tf.keras.models.Sequential):
  def __init__(self):
    super().__init__()
    self.model = tf.keras.models.Sequential()


  def build(self):
    for layer in self.model.layers:
      self.model.layer.trainable = False
    self.model.add(baseModel)
    self.model.add(AveragePooling2D(pool_size=(7, 7)))
    self.model.add(Flatten())
    self.model.add(BatchNormalization())
    self.model.add(Dense(256, activation= 'relu'))
    self.model.add(Dropout(0.5))
    self.model.add(Dense(2,activation='softmax'))

  def summary(self):
    return self.model.summary()

  def compile(self):
    self.model.compile(optimizer="adam",loss="binary_crossentropy",metrics =["accuracy"])

  def fit(self, X_train, y_train, batch_size, epochs, callbacks):
    self.model.fit(X_train, y_train, batch_size, epochs, callbacks=callbacks)

  def evaluate(self, X_test, y_test):
    evaluation = self.model.evaluate(X_test, y_test)
    print("Total loss: ", evaluation[0])
    print("Model accuracy: ", evaluation[1])

  def predict(self, X_test):
    return self.model.predict(X_test)

  def save(self, name= "model"):
    self.model.save(f"{name}.h5")

**PROCESSING DATA**

In [4]:
cat_path = "/kaggle/input/dogcatclassification/Cat_Dog_Classification_DataSet/Cat"
dog_path = "/kaggle/input/dogcatclassification/Cat_Dog_Classification_DataSet/Dog"

In [5]:
data = []
labels = []

In [6]:
#Preprocessing data, read images
def data_preprocessing(path, label):
    files = os.listdir(path) 
    for file in files:
        img = cv2.imread(path+"/"+file)
        if img is not None: 
            img = cv2.resize(img, dsize = (224,224))
            data.append(img)
            labels.append(label)

In [7]:
data_preprocessing(cat_path, 0)
data_preprocessing(dog_path, 1)

Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 128 extraneous bytes before marker 0xd9


In [8]:
#Convert dataset and labels to arrays
data_arr = np.array(data).astype(np.float32)
labels_arr = np.array(labels).astype(np.float32)

In [9]:
#normalized dataset
data_norm = data_arr/255

In [10]:
#Splitting data
X_train, X_test, y_train, y_test = train_test_split(data_norm, labels_arr, test_size=0.20, stratify=labels, random_state=42)

In [11]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [12]:
#Early Stopping if there is no improvement 
callbacks = EarlyStopping(monitor='accuracy', patience=10, restore_best_weights= True)

**TRAINING MODEL**

In [13]:
model = Model()
model.build()
model.compile()
history = model.fit(X_train, y_train, batch_size= 128, epochs= 30, callbacks= callbacks)

Epoch 1/30
25/25 [==============================] - 356s 14s/step - loss: 0.1999 - accuracy: 0.9345
Epoch 2/30
25/25 [==============================] - 323s 13s/step - loss: 0.0568 - accuracy: 0.9812
Epoch 3/30
25/25 [==============================] - 347s 14s/step - loss: 0.0224 - accuracy: 0.9934
Epoch 4/30
25/25 [==============================] - 320s 13s/step - loss: 0.0084 - accuracy: 0.9975
Epoch 5/30
25/25 [==============================] - 341s 14s/step - loss: 0.0383 - accuracy: 0.9897
Epoch 6/30
25/25 [==============================] - 322s 13s/step - loss: 0.0547 - accuracy: 0.9784
Epoch 7/30
25/25 [==============================] - 346s 14s/step - loss: 0.0239 - accuracy: 0.9922
Epoch 8/30
25/25 [==============================] - 327s 13s/step - loss: 0.0128 - accuracy: 0.9956
Epoch 9/30
25/25 [==============================] - 341s 14s/step - loss: 0.0246 - accuracy: 0.9903
Epoch 10/30
25/25 [==============================] - 322s 13s/step - loss: 0.0082 - accuracy: 0.9969

In [14]:
model.evaluate(X_test, y_test)

25/25 [==============================] - 25s 947ms/step - loss: 1.7942 - accuracy: 0.8421
Total loss:  1.794249415397644
Model accuracy:  0.8421052694320679


In [15]:
pred = model.predict(X_test)

25/25 [==============================] - 13s 483ms/step


In [16]:
print(classification_report(y_test.argmax(axis=1), pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.95      0.72      0.82       399
           1       0.78      0.96      0.86       399

    accuracy                           0.84       798
   macro avg       0.86      0.84      0.84       798
weighted avg       0.86      0.84      0.84       798



In [17]:
model.save("MobileNet")

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
